# 메모리 기능을 통한 챗봇 구현

## 환경 설정 및 OpenAI LLM 준비 
* 환경 변수(`.env` 파일)에서 API Key 로딩
* 개발 환경에서는 `gpt-4o-mini` 또는 `gpt-3.5-turbo`

In [ ]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

# .env 파일 로드
load_dotenv()

open_api_key = os.getenv("OPENAI_API_KEY")
print(f"{open_api_key[:9]}***")

# OpenAI LLM 준비
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
print(llm.model_name)

## 역할 기반 프롬프트 템플릿 정의
- `system` 역할: LLM에게 '명탐정 코난 전문가' 역할 부여
- `MessagesPlaceholder` : 프롬프트에서 memory가 삽입될 위치 지정
- `human` 역할: 사용자 질문 삽입

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# 프롬프트 템플릿 정의
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 '명탐정 코난' 전문가입니다. 대화의 흐름을 기억하며 친절하게 답변하세요."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}")
])

## 메모리 설정
메시지 기록 저장 방식 정의

### `langchain.memory` 모듈의 핵심 클래스
|주요 클래스|설명|
|---|---|
|`ChatMessageHistory`|(`현재 코드 방식`) 메모리 내(in-memory) 리스트에 메시지를 순서대로 저장하는 가장 단순한 형태|
|`ConversationBufferMemory`|`ChatMessageHistory`를 감싸서, 대화 기록 전체를 하나의 긴 문자열(버퍼)로 만들어 프롬프트에 전달|
|`ConversationBufferWindowMemory`|대화 기록 중, 가장 최근의 n개만 저장하고 나머지는 제외|
|`ConversationSummaryMemory`|대화가 진행됨에 따라, LLM을 사용해 이전 대화 내용을 요약하며 저장|
|`VectorStoreRetrieverMemory`|대화 내용을 벡터(숫자 배열)로 변환하여 DB에 저장하고, 현재 질문과 의미적으로 가장 관련성 높은 과거 대화를 검색|

### 영구 저장소(Persistent Storage) 연동시
|클래스 (from ...)|연동 DB|
|---|---|
|`RedisChatMessageHistory (langchain_community.chat_message_histories)`|Redis|
|`SQLChatMessageHistory (langchain_community.chat_message_histories)`|PostgreSQL, MySQL, SQLite 등|


In [ ]:
from langchain.memory import ChatMessageHistory

# 실제 서비스에서는 Redis나 DB 같은 영구 저장소를 사용
store = {}


# 특정 사용자의 대화 기록을 가져오는 함수 정의
def get_session_history(session_id: str) -> ChatMessageHistory:
    """세션 ID에 해당하는 메시지 기록을 가져오거나 새로 생성합니다."""

    if session_id not in store:
        # 새로운 빈 대화 기록(ChatMessageHistory) 저장
        store[session_id] = ChatMessageHistory()

    return store[session_id]

## LCEL 체인 파이프라인 구성

### `chain: Runnable = prompt | llm` (기본 체인 조립)
- `Runnable`은 이 chain이 LangChain에서 실행 가능한 단위임을 나타내는 타입 힌트

### `RunnableWithMessageHistory(...)` (체인에 기억력 결합)
- `RunnableWithMessageHistory` 작동 방식
    1. 기록 조회: session_id를 이용해 get_session_history 함수를 호출하여 해당 사용자의 과거 대화 기록을 조회
    2. 입력 조립: 가져온 과거 기록을 history_messages_key(chat_history)에, 사용자의 새 질문을 input_messages_key(question)에 담아 prompt | llm 체인에 전달할 완벽한 입력값을 생성
    3. 답변 생성: prompt | llm 체인이 이 입력값을 받아 AI의 답변을 생성
    4. 기록 저장: 생성된 AI의 답변과 사용자의 질문을 다시 get_session_history가 반환한 객체에 추가하여 최신 대화 내용을 저장(업데이트)
    5. 최종 반환: 생성된 AI의 답변만을 사용자에게 최종적으로 반환

In [ ]:
from langchain_core.runnables import Runnable
from langchain_core.runnables.history import RunnableWithMessageHistory

# 기본 체인(Chain) 조립 - 파이프(|)) 연산자를 사용하여 프롬프트와 LLM을 연결
chain: Runnable = prompt | llm

# 체인에 기억력(메모리) 결합
chain_with_memory = RunnableWithMessageHistory(
    chain,                                # 기억력을 부여할 기본 체인
    get_session_history,                  # 대화 기록을 가져올 함수 (3단계에서 만듦)
    input_messages_key="question",        # 사용자 질문을 식별할 이름
    history_messages_key="chat_history",  # 대화 기록을 식별할 이름
)

## 체인과 UI 구성
### Gradio와 LangChain 연결 함수
- `history = history or []` : Gradio의 Chatbot 컴포넌트는 history가 비어있을 경우(None을 전달) 빈 리스트로 초기화
- `result = chain_with_memory.invoke(..)` : 챗봇 엔진 체인을 호출하고 대화 기록을 사용할 수 있게 config 부분에 session_id를 전달

In [ ]:
# Gradio와 LangChain을 연결할 '브릿지' 함수
def chat_fn(user_input, history, session_id: str):
    """Gradio 인터페이스를 위한 채팅 함수"""

    history = history or []

    result = chain_with_memory.invoke(
        {"question": user_input},
        config={"configurable": {"session_id": session_id}},
    )
    
    # (사용자 입력, 챗봇 답변) 쌍을 history 리스트에 추가
    history.append((user_input, result.content))
    
    # 업데이트된 history 리스트를 반환합니다. 이 값이 Chatbot 컴포넌트에 전달
    return history

### Gradio의 세션 관리
`session_id_state = gr.State(lambda: str(uuid.uuid4()))` 작동 원리
1. `gr.State`의 초기화 규칙:
    - gr.State의 value 인자로 **함수(Function)나 lambda**를 전달
    - Gradio는 이 함수를 세션이 처음 시작될 때 단 한 번만 호출하여 그 반환 값을 초기값으로 사용
2. `lambda`의 역할
    - 여기서 lambda는 타입 힌트가 아니라, **이름 없는 일회용 함수(Anonymous Function)**
    - lambda를 사용하는 이유는 코드 실행 시점을 제어 : `gr.State`는 함수 객체를 받아두었다가, 새로운 세션이 시작될 때마다 그 함수를 호출

In [ ]:
import gradio as gr
import uuid

# Gradio UI 레이아웃 설계
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("### 🕵️‍♂️ 명탐정 코난 전문가와 대화해보세요!")
    chatbot = gr.Chatbot(label="명탐정 코난 전문가 챗봇", height=400)
    txt = gr.Textbox(placeholder="예: 검은 조직의 보스는 누구야?", show_label=False)
    
    # 사용자별 고유 ID를 저장하는 상태(State) 값
    session_id_state = gr.State(lambda: str(uuid.uuid4()))

    # '입력'과 '챗봇 엔진'과 '출력'을 연결
    txt.submit(
        chat_fn,  # Enter 키를 누르면 실행될 함수 (챗봇 로직)
        inputs=[txt, chatbot, session_id_state], # chat_fn에 전달될 입력값들
        outputs=[chatbot], # chat_fn이 반환한 결과로 업데이트될 출력 컴포넌트
    )

    # 입력창 초기화
    txt.submit(lambda: "", None, txt)

# 웹 서버 실행
demo.launch()

-----
** End of Documents **